In [40]:
import pandas as pd
import numpy as np

In [41]:
filename = 'example_data.xlsx'
xls = pd.ExcelFile(filename)

sheets = {}
for sheet_name in xls.sheet_names:
    sheets[sheet_name] = pd.read_excel(xls, sheet_name, engine='openpyxl')

heaters = sheets['Heaters']
coolers = sheets['Coolers']
HU = sheets['Hot Utilities']
CU = sheets['Cold Utilities']

In [42]:
heaters

,Name,Duty,Tin,Tout,Fcp
0,H1,5600,700,420,20
1,H2,11600,620,330,40
2,H3,10500,480,330,70
3,H4,4700,380,330,94


In [43]:
for index, row in heaters.iterrows():
    print((row['Tin'], row['Tout']))

(700, 420)
(620, 330)
(480, 330)
(380, 330)


In [44]:
def generate_table(heaters, coolers, HU, CU, HRAT=10):
    hlines = set()
    clines = set()
    for index, row in heaters.iterrows():
        Thot = row['Tin']
        hlines.add(Thot)

        clines.add(Thot-HRAT)
        Thot = row['Tout']
        hlines.add(Thot)
        clines.add(Thot-HRAT)

    for index, row in coolers.iterrows():
        Tcold = row['Tin']
        clines.add(Tcold)
        hlines.add(Tcold+HRAT)

        Tcold = row['Tout']
        clines.add(Tcold)
        hlines.add(Tcold+HRAT)

    max_ht = max(hlines)
    min_ct = min(clines)

    for index, row in HU.iterrows():
        if row['TEMP'] < max_ht:
            Thot = row['TEMP']
            hlines.add(Thot)
            clines.add(Thot-HRAT)

    for index, row in CU.iterrows():
        if row['TEMP'] > min_ct:
            Tcold = row['TEMP']
            clines.add(Tcold)
            hlines.add(Tcold+HRAT)

    hlines = sorted(hlines)
    clines = sorted(clines)
    interval = np.zeros((len(hlines), 2))
    for i in range(len(hlines)):
        assert(hlines[i] == clines[i] + HRAT)
        interval[i][0] = hlines[i]
        interval[i][1] = clines[i]
    return interval

In [45]:
interval_table = generate_table(heaters, coolers, HU, CU, HRAT=10)
interval_table

array([[310., 300.],
       [330., 320.],
       [380., 370.],
       [385., 375.],
       [410., 400.],
       [420., 410.],
       [480., 470.],
       [510., 500.],
       [620., 610.],
       [660., 650.],
       [700., 690.]])

In [46]:
def fill_table(interval_table, heaters, coolers):
    heat_tab = np.zeros(((len(interval_table)-1), len(heaters)))
    for index, row in heaters.iterrows():
        Tout = row['Tout']
        Tin = row['Tin']
        Tlo = Tout
        Tup = Tin
        for i in range(1, len(interval_table[1:])+1):
            value = 0
            int_up = interval_table[i][0]
            int_lo = interval_table[i-1][0]
            if Tlo > int_up:
                value = 0
            elif Tup < int_lo:
                value = 0
            else:
                # upper = min(Tup, int_up)
                # lower = max(Tlo, int_lo)
                # value = (upper - lower) * row['Fcp']
                value = (int_up - int_lo) * row['Fcp']
            heat_tab[i-1][index] = value

    cool_tab = np.zeros(((len(interval_table)-1), len(coolers)))
    for index, row in coolers.iterrows():
        Tout = row['Tout']
        Tin = row['Tin']
        Tlo = Tin
        Tup = Tout
        for i in range(1, len(interval_table[1:])+1):
            value = 0
            int_up = interval_table[i][1]
            int_lo = interval_table[i-1][1]
            if Tup < int_lo:
                value = 0
            elif Tlo > int_up:
                value = 0
            else:
                upper = min(Tup, int_up)
                lower = max(Tlo, int_lo)
                value = (upper - lower) * row['Fcp']
            cool_tab[i-1][index] = value

    return heat_tab, cool_tab


In [47]:
heat_tab, cool_tab = fill_table(interval_table, heaters, coolers)

In [48]:
heat_tab

array([[   0.,  800., 1400., 1880.],
       [   0., 2000., 3500., 4700.],
       [   0.,  200.,  350.,  470.],
       [   0., 1000., 1750.,    0.],
       [ 200.,  400.,  700.,    0.],
       [1200., 2400., 4200.,    0.],
       [ 600., 1200., 2100.,    0.],
       [2200., 4400.,    0.,    0.],
       [ 800., 1600.,    0.,    0.],
       [ 800.,    0.,    0.,    0.]])

In [49]:
cool_tab

array([[ 1000.,     0.],
       [ 2500.,     0.],
       [  250.,     0.],
       [ 1250., 18000.],
       [  500.,     0.],
       [ 3000.,     0.],
       [ 1500.,     0.],
       [ 5500.,     0.],
       [ 2000.,     0.],
       [    0.,     0.]])